In [3]:
import torch
from transformers import VitsTokenizer, VitsModel, set_seed

import pybangla
nrml = pybangla.Normalizer()

model_path = "samikhan121/vits_mms_ft_fahim-small_ft_nctb-sp1"
tokenizer = VitsTokenizer.from_pretrained(model_path)
model = VitsModel.from_pretrained(model_path).to("cuda").eval()

set_seed(456)  # make deterministic

In [11]:
text = "রাহিম ক্লাস ওয়ান এ ১ম, এন্ড বাসার ক্লাস এ ৩৩ তম, সে জন্য ২০৩০ শতাব্দীতে ¥২০৩০.১২৩৪ দিতে হয়েছে"
text = "My phone number is 01834407262"

text = nrml.text_normalizer(text)

print(text)

My phone number is জিরো ওয়ান এইট থ্রি ডাবল ফোর জিরো সেভেন টু সিক্স টু


In [4]:
def inference(raw_text):
    normalized_text = nrml.text_normalizer(raw_text)
    inputs = tokenizer(normalized_text, return_tensors="pt")
    outputs = model(inputs["input_ids"].to("cuda"))
    waveform = outputs.waveform[0].to("cpu").detach().numpy()
    return waveform

In [22]:
# text = "১৯৯৬ সালের ৬ সেপ্টেম্বর রণ ভ্রমণ পরিকল্পনা করছি ২০৩০ সালের ৬ সেপ্টেম্বর"
text = """
উপসচিব পদে পদোন্নতির ক্ষেত্রে প্রশাসন ক্যাডারের কর্মকর্তাদের জন্য ৫০ শতাংশ ও অন্য ক্যাডারের জন্য ৫০ শতাংশ করা বিষয়ে জনপ্রশাসন সংস্কার কমিশনের সুপারিশ নিয়ে প্রশাসন ক্যাডারের কর্মকর্তাদের মধ্যে তীব্র আপত্তি উঠেছে। 
তাঁরা বলছেন, এ সুপারিশ তাঁরা মানবেন না। 
অন্যদিকে শিক্ষা ও স্বাস্থ্যকে ক্যাডারে অন্তর্ভুক্ত না রেখে আলাদা কমিশনে রাখার যে সুপারিশ করা হচ্ছে, তা নিয়ে শিক্ষা ক্যাডারের কর্মকর্তারাও আপত্তি জানাচ্ছেন।
"""
waveform = inference(text)

In [23]:
from IPython.display import Audio

Audio(waveform, rate=model.config.sampling_rate)